## Model Data Preparation

Diese Data Preparation gilt für jede Warengruppe. Vorher immer ausführen, bevor eigens Modell durchgelaufen wird.

Basiert auf eine Kopie von neural_net_data_preparation.ipynb mit unseren Daten.

In [57]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Import Data
data = pd.read_csv('/workspaces/DS_ML_Gr_1.5/3_Neuronal_Model/Imputed_Data_KNN.csv')
data.head()  # Print first few rows to verify

,Inflationsrate,Heimspiel,Weihnachtsmarkt,Markt,Faehrverkaehr,Kreuzfahrverkehr,Temperatur,Monat,Frühling,Sommer,...,Sonnenschein,Tageslaenge (dezimal),KielerWoche,Werktag,Datum,Umsatz,Warengruppe,Temp_average,Temp_cold,Temp_warm
0,1.53,0.0,0.0,0.0,1576718.0,419447.0,17.8375,7.0,0.0,1.0,...,0.350877,17.1,0.0,1.0,2013-07-01,148.828353,1.0,1.0,0.0,0.0
1,1.53,0.0,0.0,0.0,1576718.0,419447.0,17.8375,7.0,0.0,1.0,...,0.350877,17.1,0.0,1.0,2013-07-01,535.856285,2.0,1.0,0.0,0.0
2,1.53,0.0,0.0,0.0,1576718.0,419447.0,17.8375,7.0,0.0,1.0,...,0.350877,17.1,0.0,1.0,2013-07-01,201.198426,3.0,1.0,0.0,0.0
3,1.53,0.0,0.0,0.0,1576718.0,419447.0,17.8375,7.0,0.0,1.0,...,0.350877,17.1,0.0,1.0,2013-07-01,65.890169,4.0,1.0,0.0,0.0
4,1.53,0.0,0.0,0.0,1576718.0,419447.0,17.8375,7.0,0.0,1.0,...,0.350877,17.1,0.0,1.0,2013-07-01,317.475875,5.0,1.0,0.0,0.0


In [58]:
# Define categorical features
categorical_features = ['Heimspiel',
                        'Weihnachtsmarkt',
                        'Markt',
                        'Frühling',
                        'Sommer',
                        'Herbst',
                        'Winter',
                        'Temp_warm',
                        'Temp_cold',
                        'Temp_average',
                        'Monday',
                        'Tuesday',
                        'Wednesday',
                        'Thursday',
                        'Friday',
                        'Saturday',
                        'Sunday',
                        'Schulferien',
                        'Semesterferien',
                        'Feiertage',
                        'KielerWoche',
                        'Werktag']



# Inspect data types and unique values for categorical columns
print(data[categorical_features].dtypes)
print("Unique Values:\n",data[categorical_features].apply(lambda x: x.unique()))

# Ensure categorical columns are treated as categories
for col in categorical_features:
    data[col] = data[col].astype('category')

# Encode categorical variables using pd.get_dummies
features = pd.get_dummies(data[categorical_features], drop_first=True, dtype=int)

# Include any numeric columns that are not categorical
features['Inflationsrate'] = data['Inflationsrate']
features['Temperatur'] = data['Temperatur']
features['Niederschlag'] = data['Niederschlag']
features['Schneehoehe'] = data['Schneehoehe']
features['Sonnenschein'] = data['Sonnenschein']
features['Warengruppe'] = data['Warengruppe']
features['Datum'] = data['Datum']
features['Kreuzfahrverkehr'] = data['Kreuzfahrverkehr']
features['Faehrverkaehr'] = data['Faehrverkaehr']
features['Monat'] = data['Monat']


# Construct the prepared data set including the dependent variable --> Umsatz
prepared_data = pd.concat([data[['Umsatz']], features], axis=1)


# Wenn Data imputation fertig ist, #wegnehmen

# Display the shape of the prepared data set
#print(prepared_data.shape)
# Display the first few rows of the prepared data set
#prepared_data.head()

Heimspiel          float64
Weihnachtsmarkt    float64
Markt              float64
Frühling           float64
Sommer             float64
Herbst             float64
Winter             float64
Temp_warm          float64
Temp_cold          float64
Temp_average       float64
Monday             float64
Tuesday            float64
Wednesday          float64
Thursday           float64
Friday             float64
Saturday           float64
Sunday             float64
Schulferien        float64
Semesterferien     float64
Feiertage          float64
KielerWoche        float64
Werktag            float64
dtype: object
Unique Values:
    Heimspiel  Weihnachtsmarkt  Markt  Frühling  Sommer  Herbst  Winter  \
0        0.0              0.0    0.0       0.0     1.0     0.0     0.0   
1        1.0              1.0    1.0       1.0     0.0     1.0     1.0   

   Temp_warm  Temp_cold  Temp_average  ...  Wednesday  Thursday  Friday  \
0        0.0        0.0           1.0  ...        0.0       0.0     0.0   
1  

In [59]:
#Splitting des Datensets in die jeweiligen Warengruppen

#Bennenung der Warengruppne für dataframe Namen
warengruppe_namen = {
    1: 'Brot',
    2: 'Broetchen',
    3: 'Crossaint',
    4: 'Konditorei',
    5: 'Kuchen',
    6: 'Saisonbrot'
}

# Ursprüngliche DataFrames filtern nach Warengruppe
warengruppe_dataframes = {}
for i, name in warengruppe_namen.items():
    var_name = f"df_{name}_W{i}"  # Name erstellen nach: df_Brot_W1
    warengruppe_dataframes[var_name] = prepared_data[
        (prepared_data['Warengruppe'] == i) &
        (prepared_data['Datum'] <= '2018-07-31')
    ]

# Daten bis 2019-07-31 unabhängig der Warengruppe hinzufügen
new_data = prepared_data[
    (prepared_data['Datum'] > '2018-07-31') &
    (prepared_data['Datum'] <= '2019-08-31')
]

for var_name, df in warengruppe_dataframes.items():
    updated_df = pd.concat([df, new_data], ignore_index=True)
    globals()[var_name] = updated_df



# Ergebnisse ausgeben
print("df_Brot_W1:")
print(df_Brot_W1.tail()) #Luisa

print("df_Broetchen_W2:")
print(df_Broetchen_W2.tail()) #Luisa

print("df_Crossaint_W3:")
print(df_Crossaint_W3.tail()) #Nina

print("df_Konditorei_W4:")
print(df_Konditorei_W4.tail()) #Wiebke

print("df_Kuchen_W5:")
print(df_Kuchen_W5.tail()) #Nina

print("df_Saisonbrot_W6:")
print(df_Saisonbrot_W6.tail()) #Wiebke



df_Brot_W1:
      Umsatz  Heimspiel_1.0  Weihnachtsmarkt_1.0  Markt_1.0  Frühling_1.0  \
2179     NaN              1                    0          0             0   
2180     NaN              0                    0          0             0   
2181     NaN              0                    0          0             0   
2182     NaN              0                    0          0             0   
2183     NaN              0                    0          0             0   

      Sommer_1.0  Herbst_1.0  Winter_1.0  Temp_warm_1.0  Temp_cold_1.0  ...  \
2179           1           0           0              1              0  ...   
2180           1           0           0              0              0  ...   
2181           1           0           0              1              0  ...   
2182           1           0           0              0              0  ...   
2183           1           0           0              0              0  ...   

      Inflationsrate  Temperatur  Niederschlag  Sc

#### W1_Brot

In [60]:
data = df_Brot_W1

# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
test_end_date='2019-07-30'

#Keep only wanted features
data=data[['Umsatz',
           'Datum',
           'Warengruppe',
            #Hier Features hinzufügen und Namen anpassen:
           'Sommer_1',
            'Winter_1',
            'Temp_average_1.0',
            'Temp_warm_1.0',
            'Temp_cold_1.0',
            'Feiertage_1.0',
            'Schulferien_1.0',
            'Semesterferien_1.0',
            'Monday_1',
            'Tuesday_1',
            'Wednesday_1',
            'Thursday_1',
            'Friday_1',
            'Saturday_1',
            'Sunday_1',
            'Weihnachtsmarkt_1.0',
            'Markt_1.0',
            'Heimspiel_1.0',
            'KielerWoche_1.0']]

# Convert to datetime if not already
data['Datum'] = pd.to_datetime(data['Datum'])

# Split the data based on the date thresholds
training_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[(data['Datum'] > validation_end_date) & (data['Datum'] <= test_end_date)]

# Separating features and labels
training_features = training_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
validation_features = validation_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
test_features = test_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)




KeyError: "['Sommer_1', 'Winter_1', 'Monday_1', 'Tuesday_1', 'Wednesday_1', 'Thursday_1', 'Friday_1', 'Saturday_1', 'Sunday_1'] not in index"

In [61]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data//W1_Brot_imputation"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")

#### W2_Broetchen

In [62]:
data = df_Broetchen_W2

# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
test_end_date='2019-07-30'

#Keep only wanted features
data=data[['Umsatz',
           'Datum',
           'Warengruppe',
            #Hier Features hinzufügen und Namen anpassen:
           'Weihnachtsmarkt_1.0','Temperatur' ,'Friday_1.0','Sonnenschein','Werktag_1.0']]

# Convert to datetime if not already
data['Datum'] = pd.to_datetime(data['Datum'])

# Split the data based on the date thresholds
training_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[(data['Datum'] > validation_end_date) & (data['Datum'] <= test_end_date)]

# Separating features and labels
training_features = training_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
validation_features = validation_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
test_features = test_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Training features dimensions: (1462, 5)
Validation features dimensions: (357, 5)
Test features dimensions: (364, 5)

Training labels dimensions: (1462, 1)
Validation labels dimensions: (357, 1)
Test labels dimensions: (364, 1)


/tmp/ipykernel_22220/3320380203.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Datum'] = pd.to_datetime(data['Datum'])


In [63]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data//W2_Broetchen_imputation"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")

#### W3_Croissants

In [64]:
data = df_Crossaint_W3

# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
test_end_date='2019-07-30'

#Keep only wanted features
data=data[['Umsatz',
           'Datum',
           'Warengruppe',
            #Hier Features hinzufügen und Namen anpassen:
           'Sommer_1.0',
           'Winter_1.0',
           'Temp_cold_1.0',
           'Temp_warm_1.0',
           'Temp_average_1.0',
           'Monday_1.0',
           'Wednesday_1.0',
           'Thursday_1.0',
           'Saturday_1.0',
           'Sunday_1.0',
           'Schulferien_1.0',
           'Semesterferien_1.0',
           'Feiertage_1.0',
           'Markt_1.0',   
           'Sonnenschein',
           'Niederschlag',
           #'Tageslaenge (dezimal)',
           'Weihnachtsmarkt_1.0']]

# Convert to datetime if not already
data['Datum'] = pd.to_datetime(data['Datum'])

# Split the data based on the date thresholds
training_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[(data['Datum'] > validation_end_date) & (data['Datum'] <= test_end_date)]

# Separating features and labels
training_features = training_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
validation_features = validation_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
test_features = test_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Training features dimensions: (1462, 17)
Validation features dimensions: (357, 17)
Test features dimensions: (364, 17)

Training labels dimensions: (1462, 1)
Validation labels dimensions: (357, 1)
Test labels dimensions: (364, 1)


/tmp/ipykernel_22220/640115596.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Datum'] = pd.to_datetime(data['Datum'])


In [65]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data//W3_Croissants_imputation"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")

#### W4_Konditorei

In [66]:
data = df_Konditorei_W4

# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
test_end_date='2019-07-30'

#'Inflationsrate','Heimspiel','Markt', 'Temperatur', 'Schulferien','Sommer','Herbst','Temp_cold','Saturday','KielerWoche',
          #  'Feiertage', 'Niederschlag','Schneehoehe','Sonnenschein','Weihnachtsmarkt','Monat','Semesterferien','Werktag'
          # 'Sommer','Herbst','Temp_cold','Saturday','Semesterferien','Sonnenschein','KielerWoche','Werktag'
#Keep only wanted features
data=data[['Umsatz','Datum','Warengruppe','Inflationsrate','Heimspiel_1.0','Markt_1.0', 'Temperatur', 'Schulferien_1.0',
           'Sommer_1.0','Herbst_1.0','Saturday_1.0','KielerWoche_1.0','Feiertage_1.0','Temp_cold_1.0', 'Niederschlag','Schneehoehe',
           'Sonnenschein','Weihnachtsmarkt_1.0','Semesterferien_1.0','Werktag_1.0','Monat']]

# Convert to datetime if not already
data['Datum'] = pd.to_datetime(data['Datum'])

# Split the data based on the date thresholds
training_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[(data['Datum'] > validation_end_date) & (data['Datum'] <= test_end_date)]

# Separating features and labels
training_features = training_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
validation_features = validation_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)
test_features = test_data.drop('Umsatz', axis=1).drop('Datum', axis=1).drop('Warengruppe', axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Training features dimensions: (1409, 18)
Validation features dimensions: (357, 18)
Test features dimensions: (364, 18)

Training labels dimensions: (1409, 1)
Validation labels dimensions: (357, 1)
Test labels dimensions: (364, 1)


/tmp/ipykernel_22220/3452932458.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Datum'] = pd.to_datetime(data['Datum'])


In [67]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data//W4_Konditorei_imputation"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")

#### W5_Kuchen

In [68]:
data = df_Kuchen_W5

# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
test_end_date='2019-07-30'

#Keep only wanted features
data=data[['Umsatz',
           'Datum',
           'Warengruppe',
            #Hier Features hinzufügen und Namen anpassen:
           'Markt_1.0',
           'Sonnenschein',
           'Niederschlag',
           #'Tageslaenge (dezimal)',
           'Sommer_1.0',
           'Herbst_1.0',
           'Winter_1.0',
           'Wednesday_1.0',
           'Friday_1.0',
           'Saturday_1.0',
           'Sunday_1.0',
           'Semesterferien_1.0',
           'Temp_average_1.0',
           'KielerWoche_1.0',
           'Werktag_1.0']]

# Convert to datetime if not already
data['Datum'] = pd.to_datetime(data['Datum'])

# Split the data based on the date thresholds
training_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[(data['Datum'] > validation_end_date) & (data['Datum'] <= test_end_date)]

# Separating features and labels
training_features = training_data.drop(['Umsatz','Datum','Warengruppe'], axis=1)
validation_features = validation_data.drop(['Umsatz','Datum','Warengruppe'], axis=1)
test_features = test_data.drop(['Umsatz','Datum','Warengruppe'], axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Training features dimensions: (1462, 14)
Validation features dimensions: (357, 14)
Test features dimensions: (364, 14)

Training labels dimensions: (1462, 1)
Validation labels dimensions: (357, 1)
Test labels dimensions: (364, 1)


/tmp/ipykernel_22220/1838783615.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Datum'] = pd.to_datetime(data['Datum'])


In [69]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data//W5_Kuchen_imputation"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")

#### W6_Saisonbrot

In [34]:
data = df_Saisonbrot_W6
print (data.columns)
# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'
test_end_date='2019-07-30'

#Keep only wanted features
#'Weihnachtsmarkt','Temperatur','Monat' ,'Friday','Sonnenschein','Werktag'
data=data[['Umsatz','Datum','Warengruppe','Weihnachtsmarkt_1.0','Temperatur' ,'Friday_1.0','Sonnenschein','Werktag_1.0','Monat']]

# Convert to datetime if not already
data['Datum'] = pd.to_datetime(data['Datum'])

# Split the data based on the date thresholds
training_data = data[data['Datum'] <= train_end_date]
validation_data = data[(data['Datum'] > train_end_date) & (data['Datum'] <= validation_end_date)]
test_data = data[(data['Datum'] > validation_end_date) & (data['Datum'] <= test_end_date)]

# Separating features and labels
training_features = training_data.drop(['Umsatz','Datum','Warengruppe'], axis=1)
validation_features = validation_data.drop(['Umsatz','Datum','Warengruppe'], axis=1)
test_features = test_data.drop(['Umsatz','Datum','Warengruppe'], axis=1)



training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]
test_labels = test_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Index(['Umsatz', 'Heimspiel_1.0', 'Weihnachtsmarkt_1.0', 'Markt_1.0',
       'Frühling_1.0', 'Sommer_1.0', 'Herbst_1.0', 'Winter_1.0',
       'Temp_warm_1.0', 'Temp_cold_1.0', 'Temp_average_1.0', 'Monday_1.0',
       'Tuesday_1.0', 'Wednesday_1.0', 'Thursday_1.0', 'Friday_1.0',
       'Saturday_1.0', 'Sunday_1.0', 'Schulferien_1.0', 'Semesterferien_1.0',
       'Feiertage_1.0', 'KielerWoche_1.0', 'Werktag_1.0', 'Inflationsrate',
       'Temperatur', 'Niederschlag', 'Schneehoehe', 'Sonnenschein',
       'Warengruppe', 'Datum', 'Kreuzfahrverkehr', 'Faehrverkaehr', 'Monat'],
      dtype='object')
Training features dimensions: (236, 6)
Validation features dimensions: (56, 6)
Test features dimensions: (364, 6)

Training labels dimensions: (236, 1)
Validation labels dimensions: (56, 1)
Test labels dimensions: (364, 1)


/tmp/ipykernel_22220/2534091180.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Datum'] = pd.to_datetime(data['Datum'])


In [35]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data//W6_Saisonbrot_imputation"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")